# Question Generator (QG) Model Using HuggingFace T5

## Install and Import Needed Library

In [ ]:
!pip install transformers datasets sentencepiece

In [ ]:
!pip install transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=806845b3c46655f96a8b89d10fb6255a6033e511b644b87017646ee65df3d28e
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [1]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6f8df877fa72ffd3479a3a80ed34fd1821e618ce2d033ab1673b82880e861a3e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [3]:
from google.colab import drive
from google.colab import files
import pandas as pd
import random
from datasets import Dataset
import torch
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from rouge_score import rouge_scorer

## Data Loading

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/Semester 6 (Coding Camp 2025 powered by DBS Foundation + Asdos)/Coding Camp/Capstone/kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d mintupsidup/squad-variated-indo
!unzip squad-variated-indo.zip

Dataset URL: https://www.kaggle.com/datasets/mintupsidup/squad-variated-indo
License(s): apache-2.0
  0% 0.00/103M [00:00<?, ?B/s]
100% 103M/103M [00:00<00:00, 1.33GB/s]
Archive:  squad-variated-indo.zip
  inflating: dev-v2.0.json           
  inflating: glove_50d_Indo.txt      
  inflating: train-v2.0.json         
  inflating: train_df_variated_new.csv  
  inflating: valid_df_variated_new.csv  


In [7]:
train = pd.read_json('train-v2.0.json', orient='column')
dev = pd.read_json('dev-v2.0.json', orient='column')

init_df = pd.concat([train, dev], ignore_index=True)
init_df

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'context'..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'context': '..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...
...,...,...
472,v2.0,"{'title': 'Islamism', 'paragraphs': [{'context..."
473,v2.0,"{'title': 'Imperialism', 'paragraphs': [{'cont..."
474,v2.0,"{'title': 'Warsaw', 'paragraphs': [{'context':..."
475,v2.0,"{'title': 'French_and_Indian_War', 'paragraphs..."


In [8]:
init_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   version  477 non-null    object
 1   data     477 non-null    object
dtypes: object(2)
memory usage: 7.6+ KB


## Data Preparation

### Extract Features

In [9]:
contexts, questions, answers = [], [], []

for data in init_df['data']:
    for para in data['paragraphs']:
        context = para['context']
        for all_questions in para['qas']:
            if not all_questions.get("is_impossible", False):
                question = all_questions['question']
                answer_list = all_questions['answers']
                if len(answer_list) > 0:
                    answer = answer_list[0]['text']
                    questions.append(question)
                    contexts.append(context)
                    answers.append(answer)

print(f"Total data pertanyaan: {len(questions)}")

Total data pertanyaan: 92749


In [10]:
df = pd.DataFrame({
    "context": contexts,
    "question": questions,
    "answer": answers
})

df

,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce mulai menjadi populer?,pada akhir 1990-an
1,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Daerah mana saja yang bersaing dengan Beyonce ...,menyanyi dan
2,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce meninggalkan Destiny's Child dan...,2003
3,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Di kota dan negara mana Beyonce tumbuh?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Pada dekade mana Beyonce menjadi terkenal?,akhir 1990-an
...,...,...,...
92744,"Pound-force memiliki mitra metrik, kurang umum...",Apa istilah metrik yang kurang digunakan dari ...,kilogram-force
92745,"Pound-force memiliki mitra metrik, kurang umum...",Apa daya kilogram kadang-kadang direffer ke se...,kilopond
92746,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan unit massa dalam sis...,siput
92747,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan istilah unit kekuata...,kip


### Handling Missing Values and Duplicated Data

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92749 entries, 0 to 92748
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   92749 non-null  object
 1   question  92749 non-null  object
 2   answer    92749 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [12]:
print("Missing values:\n", df.isnull().sum())
print("\nDuplikasi baris:", df.duplicated().sum())

Missing values:
 context     0
question    0
answer      0
dtype: int64

Duplikasi baris: 55


In [13]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce mulai menjadi populer?,pada akhir 1990-an
1,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Daerah mana saja yang bersaing dengan Beyonce ...,menyanyi dan
2,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce meninggalkan Destiny's Child dan...,2003
3,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Di kota dan negara mana Beyonce tumbuh?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Pada dekade mana Beyonce menjadi terkenal?,akhir 1990-an
...,...,...,...
92689,"Pound-force memiliki mitra metrik, kurang umum...",Apa istilah metrik yang kurang digunakan dari ...,kilogram-force
92690,"Pound-force memiliki mitra metrik, kurang umum...",Apa daya kilogram kadang-kadang direffer ke se...,kilopond
92691,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan unit massa dalam sis...,siput
92692,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan istilah unit kekuata...,kip


In [14]:
print("Duplikasi baris:", df.duplicated().sum())

Duplikasi baris: 0


### Data Sampling, Convert to HuggingFace Dataset & Split to Train and Test

In [15]:
sample_size = 2000
df_sample = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

contexts_list = df_sample['context'].tolist()
questions_list = df_sample['question'].tolist()
answers_list = df_sample['answer'].tolist()

raw_dataset = Dataset.from_dict({
    "context": contexts_list,
    "question": questions_list,
    "answer": answers_list
})

raw_dataset = raw_dataset.train_test_split(test_size=0.2)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 400
    })
})

### Preprocessing and Tokenization

In [16]:
tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

def preprocess(text):
    input_text = f"generate question: context: {text['context']} answer: {text['answer']}"
    target_text = text["question"]

    model_inputs = tokenizer(
        input_text, max_length=512, padding="max_length", truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_text, max_length=64, padding="max_length", truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = raw_dataset.map(preprocess, batched=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

## Load and Configure T5 Model

In [17]:
t5_model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [18]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,1.082200
200,0.598100
300,0.591900
400,0.545600


TrainOutput(global_step=400, training_loss=0.7044423198699952, metrics={'train_runtime': 162.5272, 'train_samples_per_second': 9.845, 'train_steps_per_second': 2.461, 'total_flos': 974332624896000.0, 'train_loss': 0.7044423198699952, 'epoch': 1.0})

## Load Indonesian Named Entity Recognition (NER) Model

In [19]:
ner_tokenizer = AutoTokenizer.from_pretrained("cahya/bert-base-indonesian-NER")
ner_model = AutoModelForTokenClassification.from_pretrained("cahya/bert-base-indonesian-NER")

ner_pipe = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, grouped_entities=True)

tokenizer_config.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at cahya/bert-base-indonesian-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


## Functions

In [20]:
def extract_entities(text, max_entities=5):
    entities = ner_pipe(text)
    answers = []
    for entity in entities:
        if entity['entity_group'] in ['PER', 'LOC', 'DAT']:
            word = entity['word'].replace('##', '').strip()
            if word not in answers:
                answers.append(word)
        if len(answers) >= max_entities:
            break
    return answers

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t5_model.to(device)

def get_sentence_with_answer(text, answer):
    sentences = re.split(r'(?<=[.!?]) +', text)
    for sentence in sentences:
        if answer in sentence:
            return sentence
    return text

def generate_question_from_input(input_text):
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    outputs = t5_model.generate(
        inputs["input_ids"],
        max_length=64,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_questions(text, num_questions=5):
    answers = extract_entities(text, max_entities=num_questions)
    questions = []

    if answers:
        for ans in answers:
            context = get_sentence_with_answer(text, ans)
            input_text = f"generate question: context: {context} answer: {ans}"
            question = generate_question_from_input(input_text)
            questions.append(question)
            remaining_questions = num_questions - len(questions)
        if remaining_questions > 0:
            for i in range(remaining_questions):
                input_text = f"generate question: context: {text}"
                question = generate_question_from_input(input_text)
                questions.append(question)
    else:
        for _ in range(num_questions):
            input_text = f"generate question: context: {text}"
            question = generate_question_from_input(input_text)
            questions.append(question)

    return questions

## Inference Process

In [22]:
text = "Hari lahir Pancasila jatuh pada tanggal 1 Juni yang ditandai oleh pidato yang dilakukan oleh Presiden pertama Indonesia Ir Soekarno pada tanggal 1 Juni 1945 dalam sidang Dokuritsu Junbi Cosakai (Badan Penyelidik Usaha Persiapan Kemerdekaan). Dalam pidatonya pertama kali mengemukakan konsep awal Pancasila yang menjadi dasar negara Indonesia. Adapun sejarahnya berawal dari kekalahan Jepang pada perang pasifik, mereka kemudian berusaha mendapatkan hati masyarakat dengan menjanjikan kemerdekaan kepada Indonesia dan membentuk sebuah Lembaga yang tugasnya untuk mempersiapkan hal tersebut. Lembaga ini dinamai Dokuritsu Junbi Cosakai. Pada sidang pertamanya di tanggal 29 Mei 1945 yang diadakan di Gedung Chuo Sangi In (sekarang Gedung Pancasila), para anggota membahas mengenai tema dasar negara. Sidang berjalan sekitar hampir 5 hari, kemudian pada tanggal 1 Juni 1945, Soekarno menyampaikan ide serta gagasannya terkait dasar negara Indonesia, yang dinamai “Pancasila”. Panca artinya lima, sedangkan sila artinya prinsip atau asas."
questions = generate_questions(text, num_questions=5)

for i, q in enumerate(questions, 1):
    print(f"{i}. {q}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1. Kapan Presiden pertama Indonesia dan Dokuritsu Junbi Cosakai pada tanggal 1 Juni 1945?
2. Apa yang dilakukan oleh Presiden pertama Indonesia sejak tanggal 1 Juni 1945?
3. Kapan Sidang pertama Soekarno di tanggal 29 Mei 1945?
4. Kapan presiden pertama Soekarno menyampaikan pidato pertama di Indonesia pada tanggal 1 Juni 1945?
5. Kapan The Voecology diadakan?


## Evaluation

In [23]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

### Data Train 500

In [24]:
sample_data = raw_dataset['train'][500]
context_500 = sample_data['context']
reference_question_500 = sample_data['question']

print("Konteks:\n", context_500)
print("\nPertanyaan:\n", reference_question_500)

Konteks:
 Setiap spesies patogen memiliki spektrum karakteristik interaksi dengan host manusia nya. Beberapa organisme, seperti Staphylococcus atau Streptococcus, dapat menyebabkan infeksi kulit, pneumonia, meningitis, dan bahkan sepsi yang luar biasa, respon inflamasi sistemik yang menghasilkan kejutan, vasodilasi besar dan kematian. Namun organisme ini juga merupakan bagian dari flora manusia normal dan biasanya ada di kulit atau di hidung tanpa menyebabkan penyakit sama sekali. Organisme lain selalu menyebabkan penyakit pada manusia, seperti Rickettsia, yang adalah parasit intraseluler yang dapat bertumbuh dan berkembang biak hanya dalam sel - sel organisme lain. Salah satu spesies Rickettia menyebabkan tifus, sedangkan yang lain menyebabkan demam Rocky Mountain terlihat. Chlamydia, phylum parasit intraseluler lainnya, mengandung spesies - spesies yang dapat menyebabkan pneumonia, atau infeksi saluran kencing dan mungkin terlibat dalam penyakit jantung koroner. Akhirnya, beberapa sp

In [25]:
questions_500 = generate_questions(context_500, num_questions=5)
generated_question_500 = []

for i, q in enumerate(questions_500, 1):
    generated_question_500.append(q)
    print(f"{i}. {q}")

1. Apa - apa yang sering berperan dalam demam Rocky Mountain?
2. Apa yang terjadi di luar tubuh manusia?
3. Apa jenis parasit yang selalu menyebabkan penyakit manusia?
4. Apa yang diketahui oleh manusia untuk penyakit lain?
5. Apa yang menyebabkan penyakit dari berbagai spesies yang berbeza?


In [26]:
for i, q in enumerate(generated_question_500, 1):
    scores = scorer.score(reference_question_500, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.4444444444444444, recall=0.4444444444444444, fmeasure=0.4444444444444444), 'rouge2': Score(precision=0.25, recall=0.25, fmeasure=0.25), 'rougeL': Score(precision=0.4444444444444444, recall=0.4444444444444444, fmeasure=0.4444444444444444)}
2. {'rouge1': Score(precision=0.14285714285714285, recall=0.1111111111111111, fmeasure=0.125), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.14285714285714285, recall=0.1111111111111111, fmeasure=0.125)}
3. {'rouge1': Score(precision=0.25, recall=0.2222222222222222, fmeasure=0.23529411764705882), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.25, recall=0.2222222222222222, fmeasure=0.23529411764705882)}
4. {'rouge1': Score(precision=0.125, recall=0.1111111111111111, fmeasure=0.11764705882352941), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.125, recall=0.1111111111111111, fmeasure=0.11764705882352941)}

### Data Train 250

In [27]:
sample_data = raw_dataset['train'][250]
context_250 = sample_data['context']
reference_question_250 = sample_data['question']

print("Konteks:\n", context_250)
print("\nPertanyaan:\n", reference_question_250)

Konteks:
 Perjalanan dan pariwisata terus menjadi sangat penting bagi Portugal, dengan jumlah pengunjung yang diperkirakan akan meningkat secara signifikan di masa depan. Oleh karena itu, negara ini perlu berfokus pada daya tariknya, seperti kesehatan, alam, dan pedesaan, untuk tetap berada di depan para pesaingnya.

Pertanyaan:
 Bagaimana Portugal bersaing dengan daerah - daerah lain bagi wisatawan?


In [28]:
questions_250 = generate_questions(context_250, num_questions=5)
generated_question_250 = []

for i, q in enumerate(questions_250, 1):
    generated_question_250.append(q)
    print(f"{i}. {q}")

1. Apa yang telah berlaku di Portugal selama tahun berapa?
2. Siapa yang melihat jumlah pengunjung yang diperkirakan meningkat secara signifikan?
3. Berapa banyak kunjungan pengunjung ke Portugal?
4. Negara mana yang meningkatkan jadwal pelancongan?
5. Negara apa yang perlu fokus pada apa?


In [29]:
for i, q in enumerate(generated_question_250, 1):
    scores = scorer.score(reference_question_250, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.1111111111111111, recall=0.1111111111111111, fmeasure=0.1111111111111111), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.1111111111111111, recall=0.1111111111111111, fmeasure=0.1111111111111111)}
2. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
3. {'rouge1': Score(precision=0.16666666666666666, recall=0.1111111111111111, fmeasure=0.13333333333333333), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.16666666666666666, recall=0.1111111111111111, fmeasure=0.13333333333333333)}
4. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
5. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0,

### Data Train 150

In [30]:
sample_data = raw_dataset['train'][150]
context_150 = sample_data['context']
reference_question_150 = sample_data['question']

print("Konteks:\n", context_150)
print("\nPertanyaan:\n", reference_question_150)

Konteks:
 Dalam bukunya, Humanism (1997), Tony Davies menyebut kritikus ini "manusia anti-manusia." Para kritikus antihumanisme, terutama Jürgen Habermas, membantah bahwa meskipun para antihumanis mungkin menonjolkan kegagalan humanisme untuk memenuhi standar emansipasinya, mereka tidak menawarkan proyek emansipasi alternatif mereka sendiri. Lainnya, seperti filsuf Jerman Heidegger menganggap diri humanis pada model Yunani kuno, tetapi berpikir humanisme diterapkan hanya untuk "race" Jerman dan khusus untuk Nazi dan dengan demikian, dengan kata-kata Davies, adalah humanis anti-humanis. Membaca pikiran Heidegger itu sendiri sangat kontroversial; Heidegger mencakup pandangannya sendiri dan kritik Humanisme dalam Letter On Humanism. Davies mengakui bahwa setelah pengalaman mengerikan dari perang-perang abad ke-20 seharusnya tidak lagi mungkin untuk merumuskan frase seperti 'takdir manusia' atau 'bahasa manusia' tanpa kesadaran instan kebodohan dan kebrutalan mereka menyeret di belakang me

In [31]:
questions_150 = generate_questions(context_150, num_questions=5)
generated_question_150 = []

for i, q in enumerate(questions_150, 1):
    generated_question_150.append(q)
    print(f"{i}. {q}")

1. Dalam bukunya Humanism, ia menyebut para antihumanis sebagai musuh manusia seperti apa?
2. Apa kata pengkritik terkenal dengan kritikus anti-manusia?
3. Siapa yang mengkritik Humanisme?
4. Siapa yang mengkritik Humanisme?
5. Siapa yang menyatakan humanisme?


In [32]:
for i, q in enumerate(generated_question_150, 1):
    scores = scorer.score(reference_question_150, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
2. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
3. {'rouge1': Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), 'rouge2': Score(precision=0.3333333333333333, recall=0.25, fmeasure=0.28571428571428575), 'rougeL': Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665)}
4. {'rouge1': Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), 'rouge2': Score(precision=0.3333333333333333, recall=0.25, fmeasure=0.28571428571428575), 'rougeL': Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665)}
5. {'rouge1': Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), 'rouge2': Score(precision=0.3333333333333333, recall=0.25, fmeasure=0.285714285714

### Data Train 50

In [33]:
sample_data = raw_dataset['train'][50]
context_50 = sample_data['context']
reference_question_50 = sample_data['question']

print("Konteks:\n", context_50)
print("\nPertanyaan:\n", reference_question_50)

Konteks:
 Pada bulan Mei 2008 mencetak The Times beralih dari Wapping ke tanaman baru di Broxbourne di pinggiran London, dan Merseyside dan Glasgow, memungkinkan kertas diproduksi dengan warna penuh di setiap halaman untuk pertama kalinya.

Pertanyaan:
 Pada bulan Mei 2008, The Times beralih dari tanaman apa ke tanaman baru di Broxbourne, Merseyside, dan Glasgow?


In [34]:
questions_50 = generate_questions(context_50, num_questions=5)
generated_question_50 = []

for i, q in enumerate(questions_50, 1):
    generated_question_50.append(q)
    print(f"{i}. {q}")

1. Apa yang memungkinkan kertas The Times di musim panas 2008 yang digunakan di Broxbourne dan Merseyside Glasgow?
2. Apa tanaman baru di Wapping?
3. Pada bulan November 2008, The Times mencetak The Times dalam keadaan warna penuh di berbagai halaman.
4. Di mana The Times dicetak?
5. The Times mencetak anting-anting, dari tanaman di pinggiran London, di pinggiran mana?


In [35]:
for i, q in enumerate(generated_question_50, 1):
    scores = scorer.score(reference_question_50, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.5294117647058824, recall=0.5, fmeasure=0.5142857142857143), 'rouge2': Score(precision=0.125, recall=0.11764705882352941, fmeasure=0.12121212121212122), 'rougeL': Score(precision=0.35294117647058826, recall=0.3333333333333333, fmeasure=0.34285714285714286)}
2. {'rouge1': Score(precision=0.8, recall=0.2222222222222222, fmeasure=0.3478260869565218), 'rouge2': Score(precision=0.5, recall=0.11764705882352941, fmeasure=0.19047619047619047), 'rougeL': Score(precision=0.8, recall=0.2222222222222222, fmeasure=0.3478260869565218)}
3. {'rouge1': Score(precision=0.375, recall=0.3333333333333333, fmeasure=0.35294117647058826), 'rouge2': Score(precision=0.2, recall=0.17647058823529413, fmeasure=0.18750000000000003), 'rougeL': Score(precision=0.375, recall=0.3333333333333333, fmeasure=0.35294117647058826)}
4. {'rouge1': Score(precision=0.6, recall=0.16666666666666666, fmeasure=0.2608695652173913), 'rouge2': Score(precision=0.25, recall=0.058823529411764705, fmeasure=0.

### Data Train 5

In [36]:
sample_data = raw_dataset['train'][5]
context_5 = sample_data['context']
reference_question_5 = sample_data['question']

print("Konteks:\n", context_5)
print("\nPertanyaan:\n", reference_question_5)

Konteks:
 Armenia mendirikan sebuah Gereja yang masih berdiri secara independen di antara gereja Katolik maupun Gereja Ortodoks Timur, yang telah menjadi demikian pada tahun 451 AD akibat pendiriannya mengenai Konsili Chalcedon. Dewasa ini, gereja ini dikenal sebagai Gereja Rasuli Armenia, yang merupakan bagian dari persekutuan Ortodoks Timur, tidak boleh disalahartikan dengan persekutuan Ortodoks Timur. Selama gerhana politiknya belakangan, Armenia bergantung pada gereja untuk melestarikan dan melindungi identitasnya yang unik. Lokasi semula penganut Katolik Armenia adalah Echmiadzin. Akan tetapi, pergolakan yang terus - menerus terjadi, yang menjadi ciri adegan politik Armenia, membuat kekuasaan politik pindah ke tempat - tempat yang lebih aman. Pusat Gereja juga pindah ke berbagai lokasi bersama dengan otoritas politik. Oleh karena itu, ia akhirnya pindah ke Kilikia sebagai Pribadi Suci Kilikia.

Pertanyaan:
 Di mana Katolikosat Armenia pindah ke?


In [37]:
questions_5 = generate_questions(context_5, num_questions=5)
generated_question_5 = []

for i, q in enumerate(questions_5, 1):
    generated_question_5.append(q)
    print(f"{i}. {q}")

1. Apa gereja yang masih berdiri secara independen di antara gereja Katolik atau gereja Ortodoks,?
2. Echmiadzin pindah ke Kilikia dengan tujuan untuk melindungi gereja-gereja di Armenia.
3. Apa gereja yang dianggap sebagai bagian dari persekutuan Ortodoks Timur?
4. Apa yang menjadi salah satu pusat agama Katolik Armenia?
5. Kapan Gereja Rasuli Armenia baru diciptakan?


In [38]:
for i, q in enumerate(generated_question_5, 1):
    scores = scorer.score(reference_question_5, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.07142857142857142, recall=0.16666666666666666, fmeasure=0.1), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.07142857142857142, recall=0.16666666666666666, fmeasure=0.1)}
2. {'rouge1': Score(precision=0.3333333333333333, recall=0.6666666666666666, fmeasure=0.4444444444444444), 'rouge2': Score(precision=0.09090909090909091, recall=0.2, fmeasure=0.12500000000000003), 'rougeL': Score(precision=0.16666666666666666, recall=0.3333333333333333, fmeasure=0.2222222222222222)}
3. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
4. {'rouge1': Score(precision=0.1111111111111111, recall=0.16666666666666666, fmeasure=0.13333333333333333), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.1111111111111111, recall=0.16666666666666666, fmeasure=0.13333333333333333)}
5. {'ro

## Save Pretrained Model and Tokenizer

In [39]:
t5_model.save_pretrained("qg_model")
tokenizer.save_pretrained("qg_model")

('qg_model/tokenizer_config.json',
 'qg_model/special_tokens_map.json',
 'qg_model/spiece.model',
 'qg_model/added_tokens.json')